In [1]:
import numpy as np
import sympy as sp
from sympy import cos, sin

In [2]:
# Source: symbolic_fk.ipynb demo code

def rz(theta):
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,:] = [[cos(theta), -sin(theta),0,0]]
    rot[1,:] = [[sin(theta), cos(theta),0,0]]
    rot[2,2] = 1

    return rot

def rx(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[0,0] = 1
    rot[1,:] = [[0, cos(theta), -sin(theta),0]]
    rot[2,:] = [[0, sin(theta), cos(theta),0]]

    return rot


def ry(theta): 
    rot = sp.zeros(4,4)
    rot[3,3] = 1
    rot[1,1] = 1
    rot[0,:] = [[ cos(theta), 0, sin(theta),0]]
    rot[2,:] = [[-sin(theta),0, cos(theta),0]]

    return rot


def trans(vector):
    mat = sp.eye(4)
    mat[0:3,3] = vector
    return mat

def tx(dx):
    return trans([dx, 0, 0])

def ty(dy):
    return trans([0, dy, 0])

def tz(dz):
    return trans([0, 0, dz])

In [3]:
t = sp.Symbol('t')
qs = [sp.Function(f'q_{i}')(t) for i in range(6)] # Joints effect
dqs = [sp.Derivative(q) for q in qs] # Joint velocities
ls = sp.symbols(r"l_:6") # Link lengths
hs = sp.symbols(r"h_:3") # Link heights
ws = sp.symbols(r"w_:3") # Link widths
rs = sp.symbols(r"r_:6") # Link radii

In [4]:
dh_table = [
    [0, 0, 0, 0],
    [sp.pi / 2, sp.pi / 2, 0, qs[1] + ls[1]],
    [0, -sp.pi / 2, 0, qs[2] + ls[2]],
    [-sp.pi / 2 + qs[3], sp.pi / 2, 0, ls[3]],
    [sp.pi / 2 + qs[4], sp.pi / 2, 0, 0],
    [qs[5], 0, 0, ls[4]]
]

In [5]:
def dh_to_matrix(dh_row):
    theta, alpha, a, d = dh_row
    return rz(theta) * tx(a) * tz(d) * rx(alpha)

In [6]:
def forward(i, j):
    T = sp.eye(4)
    for k in range(i, j):
        T = T * dh_to_matrix(dh_table[k])
    return T

![](assets/frames.png)

In [7]:
coms = np.zeros((6, 4, 4), dtype=object)
for i in range(0, 6):
    coms[i] = forward(0, i + 1)
coms[4] = coms[4] * tz(ls[4] / 2)

In [8]:
w1 = sp.Matrix([0, 0, 0]) # local axis of joint 1
w2 = sp.Matrix([0, 0, 0]) # local axis of joint 2
w3 = sp.Matrix([0, 0, 1]) # local axis of joint 3
w4 = sp.Matrix([0, 1, 0]) # local axis of joint 4
w5 = sp.Matrix([1, 0, 0]) # local axis of joint 5

R1 = forward(0, 4)[0:3,0:3]
R2 = forward(0, 5)[0:3,0:3]
R3 = forward(0, 6)[0:3,0:3]

w3 = R1 @ w3
w4 = R2 @ w4
w5 = R3 @ w5

In [9]:
w3

Matrix([
[ sin(q_3(t))],
[-cos(q_3(t))],
[           0]])

In [10]:
Jv = np.zeros((6, 3, 5), dtype=object)
Jw = np.zeros((6, 3, 5), dtype=object)

for i in range(6):
  for j in range(3):
    for k in range(5):
      Jv[i, j, k] = sp.diff(coms[i, j, 3], qs[k])

ws = [w1, w2, w3, w4, w5]
for n in range(6):
  for j in range(n):
    Jw[n, :, j] = np.array(ws[j].tolist()).T


In [11]:
J = [sp.Matrix(i).col_join(sp.Matrix(j)) for i, j in zip(Jv, Jw)]

In [12]:
m = [1 for _ in range(6)]
I = np.array([[1 for b in ['x', 'y', 'z']] for a in ['x', 'y', 'z']])
R = np.zeros((6, 3, 3), dtype=object)
for n in range(6):
  R[n] = coms[n, :3, :3]

M = np.zeros((5, 5), dtype=object)
for i in range(6):
  M += m[i] * Jv[i].T @ Jv[i] + Jw[i].T @ R[i] @ I @ R[i].T @ Jw[i]

In [13]:
steps = 2 # number of steps to simplify
# one step was not enough for my case
for i in range(steps):
    M = sp.trigsimp(M)

In [14]:
M = sp.Matrix(M)
M

Matrix([
[0,                   0,                                                                                0,                                                                                0,                                                             0],
[0,                   5,                                                                                0,                                                                                0,                                           3*l_4*cos(q_4(t))/2],
[0,                   0,                                                                sin(2*q_5(t)) + 7, -3*l_4*sin(q_3(t) - q_4(t))/4 - 3*l_4*sin(q_3(t) + q_4(t))/4 + sin(2*q_5(t)) + 2, -3*l_4*sin(q_4(t))*cos(q_3(t))/2 + sqrt(2)*sin(q_5(t) + pi/4)],
[0,                   0, -3*l_4*sin(q_3(t) - q_4(t))/4 - 3*l_4*sin(q_3(t) + q_4(t))/4 + sin(2*q_5(t)) + 2,                        5*l_4**2*cos(2*q_4(t))/8 + 5*l_4**2/8 + sin(2*q_5(t)) + 2,                                    sqrt(2)*

In [15]:
quick_check = (M==M.T)

if quick_check == 1:
    print('Matrix is symmetric')
else:
    print('Matrix is not symmetric')

    for i in range(len(M[0])):
        for j in range(i):
            if M[i, j] != M[j, i]:
                print(f'First element: {M[i, j]}\nSecond element: {M[j, i]}')

Matrix is symmetric


In [16]:
steps = 50 # number of steps to check definiteness
flag = True
for i in range(steps):
    qsf = np.random.rand(6)
    Mf = M.subs({qs[i]: qsf[i] for i in range(6)}).evalf()
    Mf = Mf.subs({ls[i]: 1 for i in range(6)}).evalf()
    eigs = np.array(list(Mf.eigenvals().keys())[1:])
    
    if not np.all(eigs > 0):
        print('Matrix is not positive definite')
        flag = False
        break

if flag:
    print('Matrix is positive definite')

Matrix is positive definite


In [17]:
C = np.zeros((5, 5), dtype=object)
for i in range(5):
  for j in range(5):
    for k in range(5):
      cijk = 0.5 * (sp.diff(M[i,j], qs[k]) + sp.diff(M[i,k], qs[j]) - sp.diff(M[j,k], qs[i]))
      C[i, j] += cijk * dqs[k]

In [18]:
g = np.zeros(6, dtype=object)
for i in range(5):
  for k in range(5):
    g[i] -= np.dot(Jv[k, :, i], m[k] * np.array([0, 0, 9.81]))

sp.Array(g)

[0, -39.24, 0, 0, -4.905*l_4*cos(q_4(t)), 0]

In [25]:
def find_coeff(t0, tf, q0, qf, dq0, dqf):
    a = sp.Matrix(sp.symbols('a:4'))
    A = sp.Matrix([[t0**3, t0**2, t0, 1],
                   [tf**3, tf**2, tf, 1],
                   [3*t0**2, 2*t0, 1, 0],
                   [3*tf**2, 2*tf, 1, 0]])
    b = sp.Matrix([q0, qf, dq0, dqf])

    sol = sp.solve(A @ a - b, a)
    return sol

In [37]:
list(find_coeff(0, 1, 0, 1, 0, 0).values())

[-2, 3, 0, 0]

In [41]:
def gen_trajectory(t0, tf, q0, qf, dq0, dqf, steps):
    a = list(find_coeff(t0, tf, q0, qf, dq0, dqf).values())
    t = np.linspace(t0, tf, steps)
    q = a[0] * t**3 + a[1] * t**2 + a[2] * t + a[3]
    dq = a[0] * 3 * t**2 + a[1] * 2 * t + a[2]
    ddq = a[0] * 6 * t + a[1] * 2
    return q, dq, ddq

In [42]:
gen_trajectory(0, 1, 0, 1, 0, 0, 5)

(array([0, 0.156250000000000, 0.500000000000000, 0.843750000000000,
        1.00000000000000], dtype=object),
 array([0, 1.12500000000000, 1.50000000000000, 1.12500000000000, 0],
       dtype=object),
 array([6, 3.00000000000000, 0, -3.00000000000000, -6.00000000000000],
       dtype=object))

In [ ]:
# ddq = M^-1 ( -C @ dq - g + torque)

Kp = np.ones(5)
Kd = np.ones(5)

def pd_control(dt, q, dq, ddq, qd, dqd, ddqd):

    n_M = M.subs({qs[i]: q[i] for i in range(6)}).evalf()
    n_M = n_M.subs({ls[i]: 1 for i in range(6)}).evalf()
    n_M = n_M.inv()
    n_C = C.subs({qs[i]: q[i] for i in range(6)}).evalf()
    n_C = n_C.subs({ls[i]: 1 for i in range(6)}).evalf()
    n_g = g.subs({qs[i]: q[i] for i in range(6)}).evalf()
    n_g = n_g.subs({ls[i]: 1 for i in range(6)}).evalf()

    n_q = q + dq * dt
    n_dq = dq + ddq * dt

    tau = Kp * (qd - n_q) + Kd * (dqd - n_dq)

    print(tau)

    n_ddq = n_M @ (-n_C @ n_dq - n_g + tau)

In [26]:
find_coeff(0, 5, 0, 1, 0, 0)

{a0: -2/125, a1: 3/25, a2: 0, a3: 0}